In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.optimize
from sklearn import metrics, model_selection

# Diagnóstico de cancer usando Regresor Logístico

Considere el dataset de diagnóstico de cancer de mama de la Universidad de Wisconsin

In [ ]:
df = pd.read_csv('../data/cancer.csv', index_col=0)
df['diagnosis'] = df['diagnosis'].map({'M': 0, 'B': 1})
display(df.head())

El objetivo de esta actividad es entrenar un regresor logístico para clasificar las muestras como benignas y malignas en base a 30 atributos

1. Primero se describe el modelo de forma teórica y se muestra el estimador de máxima verosimilutd
1. Luego se implementa el modelo usando `numpy` y se optimize usando `scipy.optimize`
1. El desempeño del modelo se mide usando `sklearn.metrics`

# Modelo de regresión logística

Sea un problema de clasificación de $M$ observaciones $\{\vec x_i, y_i\}$ donde $\vec x_i \in \mathbb{R}^D$ (D atributos) y $y_i \in \{0, 1\}$ (clasificación binaria)

Se propone el siguiente modelo conocido como **regresión logística** con $D+1$ parámetros

$$
y_i \approx \mathcal{S} \left(\theta_0 + \sum_{j=1}^D \theta_j x_{ij}\right),
$$

donde $\mathcal{S}(z) = \frac{1}{1+\exp(-z)}$

> En lo que sigue asumiremos que las observaciones son iid y que $y_i$ sigue una distribución de Bernoulli 


Actividades:
1. Obtenga una expresión simplificada para la función de costo: máximo logaritmo de la verosimilitud 
1. Obtenga una expresión simplificada para el gradiente de la función de costo con respecto a $\theta$

## Verosimilitud

En lo que sigue llamaremos

$$
f_\theta(x_i) = \theta_0 + \sum_{j=1}^D \theta_j x_{ij}
$$


Con el supuesto iid tenemos que la verosimilitud conjunta es igual a la multiplicación de las marginales

Si además aplicamos logaritmo tenemos

$$
\log \mathcal{L}(\theta) = \sum_{i=1}^M \log p(y_i | \mathcal{S} (f_\theta(x_i)) )
$$

Luego si asumimos que la probabilidad de $y_i$ dado el modelo $\mathcal{S} (f(x_i, \theta))$ es Bernoulli entonces

$$
\begin{align}
\log p(y_i | \mathcal{S} (f_\theta(x_i) ) &= \log \mathcal{S} (f_\theta(x_i))^{y_i} (1-\mathcal{S} (f_\theta(x_i)) )^{1-y_i} \nonumber \\
&= y_i \log (\mathcal{S} (f_\theta(x_i)) ) + (1-y_i) \log(1-\mathcal{S} (f_\theta(x_i)) ) \nonumber
\end{align}
$$

y todo junto

$$
\log \mathcal{L}(\theta) = \sum_{i=1}^M y_i \log (\mathcal{S} (f_\theta(x_i)) ) + (1-y_i) \log(1-\mathcal{S} (f_\theta(x_i)) )
$$



## Derivada de la verosimilitud

La derivada de la verosimilitud con respecto a $\theta_j$ es

$$
\frac{d}{d\theta_j} \log \mathcal{L}(\theta) = \sum_{i=1}^M \left ( \frac{y_i}{\mathcal{S} (f_\theta(x_i))} - \frac{1-y_i}{1-\mathcal{S} (f_\theta(x_i))} \right) \frac{d \mathcal{S} (f(x_i, \theta))}{d f_\theta(x_i)} \frac{d f_\theta(x_i)}{d \theta_j} 
$$

donde

$$
\frac{d \mathcal{S}(z)}{dz} =  \mathcal{S}(z) ( 1 - \mathcal{S}(z))
$$

entonces

$$
\begin{align}
\frac{d}{d\theta_j} \log \mathcal{L}(\theta) &= \sum_{i=1}^M \left[ y_i - y_i\mathcal{S} (f_\theta(x_i)) - \mathcal{S} (f_\theta(x_i))  + y_i \mathcal{S} (f_\theta(x_i)) \right] \frac{d f_\theta(x_i)}{d \theta_j} \nonumber \\
&= \sum_{i=1}^M \left[ y_i  - \mathcal{S} (f_\theta(x_i)) \right] \frac{d f_\theta(x_i)}{d \theta_j} \nonumber 
\end{align}
$$

La derivada con respecto a los parámetros es 

$$
\frac{d f(x_i, \theta)}{d \theta_0} =  1
$$

y para $j>0$
$$
\frac{d f(x_i, \theta)}{d \theta_j} =  x_{ij}
$$

## Implementación del modelo en `numpy`

In [ ]:
# Modelo y optimización
def sigmoide(z):
    return 1.0/(1.0 + np.exp(-z))

def modelo(theta, X):
    f = theta[0] + np.sum(theta[1:]*X, axis=1)     
    return sigmoide(f), f

# Funciones que usaremos con scipy.optimize.minimize

def neglogverosimilitud(theta, *args):
    X, y = args
    s, f = modelo(theta, X)
    # Le agregamos un signo menos ya que quiero minimizar en lugar de maximizar
    return -np.sum(-np.logaddexp(0, -f) - (1-y)*f, axis=0)
    # La linea superior es una versión simplificada de:
    # La función logaddexp es más estable numericamente
    #return -np.sum(y*np.log(s+1e-10) + (1-y)*np.log(1-s+1e-10) )

def grad_neglogverosimilitud(theta, *args):
    X, y = args
    N = len(y)
    s, f = modelo(theta, X)
    X1 = np.concatenate((np.ones(shape=(N, 1)), X), axis=1)
    e = (y - s)
    return -np.sum(e[:, np.newaxis]*X1, axis=0)

# Entrenamiento del modelo

En primer lugar separaremos la data en conjuntos de entrenamiento y prueba usando `sklearn.model_selection`

In [ ]:
# Crear base de datos
y = df["diagnosis"].values
X = df.drop(columns=["diagnosis"]).values

# Estandarizar
X = (X - np.mean(X, axis=0))/np.std(X, axis=0)

# Partición estratificada
sss = model_selection.StratifiedShuffleSplit(n_splits=1, train_size=0.75)
train_idx, test_idx = next(sss.split(X, y))

Para entrenar el modelo usaremos la función `minimize` de `scipy.optimize` 

Usaremos `BFGS`, un método quasi-Newton que usa información de las primeras derivadas

In [ ]:
# Usaremos un callback para guardar el mejor modelo de validación
def eval_model(theta):  
    global best_theta, best_logl
    logltrain = neglogverosimilitud(theta, *(X[train_idx, :], y[train_idx]))
    logltest = neglogverosimilitud(theta, *(X[test_idx, :], y[test_idx]))
    print("Train: %0.4f, Test: %0.4f" %(logltrain, logltest))   
    if logltest < best_logl: # Guardar el mejor modelo de test
        best_theta = theta
        best_logl = logltest

# Valor inicial de theta
theta = np.random.randn(1+X.shape[1])
# Mejor valor de theta
best_theta = np.zeros(1+X.shape[1])
# Mejor valor de la verosimilitud
best_logl = np.inf
res = scipy.optimize.minimize(fun=neglogverosimilitud, x0=theta, 
                              method='BFGS', jac=grad_neglogverosimilitud, 
                              args=(X[train_idx, :], y[train_idx]),
                              callback=eval_model, tol=1e-1)

print(res.message)

# Evaluación de modelo de clasificación

La salida de este clasificador es un número en el rango $[0, 1]$

Para tomar un decisión binaria se debe seleccionar un umbral $\mathcal{T}$ tal que

$$
d_i = 
\begin{cases} 
\text{maligno} (0), & \text{si } p(y_i|\theta, \vec x_i) < \mathcal{T} \\ 
\text{benigno} (1), & \text{si } p(y_i|\theta, \vec x_i) \geq \mathcal{T}
\end{cases}
$$

Una vez seleccionado el umbral se puede contar la cantidad de 
- True positives (TP): Tumores benignos clasificados como benignos
- True negative (TN): Tumores malignos clasificados como malignos
- False positives (FP): Tumores malignos clasificados como benignos
- False negative (FN): Tumores  benignos clasificados como malignos

Estas métricas son la base para construir una "tabla de confusión" para el clasificador

|Clasificado como/En realidad era|Positivo|Negativo|
|---|---|---|
|Positivo:|TP | FP |
|Negativo:| FN | TN |


In [ ]:
display("Parametros", best_theta)
haty_train, _ = modelo(best_theta, X[train_idx, :])
haty_test, _ = modelo(best_theta, X[test_idx, :])

# Encontrando el umbral
T = np.linspace(0, 1, num=100)
f1 = np.array([metrics.f1_score(y[test_idx], haty_test > t) for t in T])

# Resultados de clasificación
display("Los resultados de clasificación en entrenamiento")
print(metrics.classification_report(y[train_idx], haty_train > T[np.argmax(f1)]))
display("Los resultados de clasificación en validación")
print(metrics.classification_report(y[test_idx], haty_test > T[np.argmax(f1)]))

display("Matriz de confusión")
cm = metrics.confusion_matrix(y[test_idx], haty_test > T[np.argmax(f1)])
for i in range(2):
    for j in range(2):
        print("Era {0} y predije {1}: {2}".format(i, j, cm[i, j]))